# Tracer Advection Stencil

#### 1. Start a parallel cluster

In [1]:
from functions import show_clusters

import ipyparallel as ipp

cluster = ipp.Cluster(engines='mpi', n=6).start_and_connect_sync()


Starting 6 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/6 [00:00<?, ?engine/s]

In [2]:
%autopx

%autopx enabled


#### 2. Configure the domain

In [3]:
%%capture
from mpi4py import MPI
import functions as func
import importlib
importlib.reload(func)


mpi_comm = MPI.COMM_WORLD
mpi_size = mpi_comm.Get_size()
mpi_rank = mpi_comm.Get_rank()

backend = 'numpy'
layout = (1, 1)
nx, ny, nz, nhalo = 101, 101, 80, 3
dimensions = {'nx': nx, 'ny': ny, 'nx1': nx+1, 'ny1': ny+1, 'nz': nz, 'nhalo': nhalo, 'tile': mpi_size, 'nxhalo': nx+2*nhalo, 'nyhalo': ny+2*nhalo}
units = {'dist': 'm', 'coord': 'degrees', 'qvapor': 'kg/kg', 'psi': 'kg/m/s', 'wind': 'm/s', 'courant': '', 'areaflux': 'm2', 'qflux': 'kg/m2', 'pressure': 'Pa'}
origins = {'halo': (0, 0), 'compute_2d': (dimensions['nhalo'], dimensions['nhalo']), 'compute_3d': (dimensions['nhalo'], dimensions['nhalo'], 0)}


configuration = func.configure_domain(layout, mpi_comm, dimensions, backend=backend)

%px:   0%|          | 0/6 [00:00<?, ?tasks/s]

#### 2a. Gather domain coordinates

In [4]:
from pace.util import Quantity
import numpy as np

dx_halo =  Quantity(configuration['grid_data'].dx.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dy_halo =  Quantity(configuration['grid_data'].dy.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dxa_halo =  Quantity(configuration['grid_data'].dxa.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dya_halo =  Quantity(configuration['grid_data'].dya.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dxc_halo =  Quantity(configuration['grid_data'].dxc.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dyc_halo =  Quantity(configuration['grid_data'].dyc.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)

lona_halo =  Quantity(configuration['grid_data'].lon_agrid.data * 180 / np.pi, ('x_halo', 'y_halo'), units['coord'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
lata_halo =  Quantity(configuration['grid_data'].lat_agrid.data * 180 / np.pi, ('x_halo', 'y_halo'), units['coord'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
lona_global = configuration['communicator'].gather(lona_halo)
lata_global = configuration['communicator'].gather(lata_halo)

#### 3. Create initial conditions on a plane

#### 3a. Create water vapor initial condition

In [5]:
if mpi_rank == 0:
    gaussian_multiplier = func.create_gaussianMultiplier(np.deg2rad(lona_global.data), np.deg2rad(lata_global.data), dimensions, center_tile=0)
else:
    gaussian_multiplier = np.zeros((dimensions['tile'], dimensions['nxhalo'], dimensions['nyhalo']))

mpi_comm.Bcast(gaussian_multiplier, root=0)
qvapor_2d = Quantity(gaussian_multiplier[mpi_rank], ('x', 'y'), units['qvapor'], origins['compute_2d'], (dimensions['nx'], dimensions['ny']), backend)


[stdout:0] Centering gaussian on lon=350.50, lat=0.50


#### 3b. Create pressure thickness initial condition

In [6]:
delp_2d = Quantity(np.ones(qvapor_2d.data.shape)*10, ('x', 'y'), units['pressure'], origins['compute_2d'], (dimensions['nx'], dimensions['ny']), backend)


#### 3c. Run coordinates through streamfunction generation

In [7]:
psi, psi_staggered = func.calculate_streamfunction_testCase1(np.deg2rad(lona_halo.data), np.deg2rad(lata_halo.data), dimensions)

psi_halo = Quantity(psi, ('x_halo', 'y_halo'), units['psi'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
psi = Quantity(psi, ('x', 'y'), units['psi'], origins['compute_2d'], (dimensions['nx'], dimensions['ny']), backend)
psi_staggered_halo = Quantity(psi_staggered, ('x_halo', 'y_halo'), units['psi'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
psi_staggered  = Quantity(psi_staggered, ('x_interface', 'y_interface'), units['psi'], origins['compute_2d'], (dimensions['nx1'], dimensions['ny1']), backend)

#### 3d. Calculate winds from streamfunction

`ua`, `va` - winds on cell centers
`uc`, `vc` - covariant winds on C-grid

In [8]:
ua, va = func.calculate_windsFromStreamfunction_grid(psi_halo, dxa_halo, dya_halo, dimensions, grid='A')

ua_2d = Quantity(ua, ('x', 'y'), units['wind'], origins['compute_2d'], (dimensions['nx'], dimensions['ny']), backend)
va_2d = Quantity(va, ('x', 'y'), units['wind'], origins['compute_2d'], (dimensions['nx'], dimensions['ny']), backend)

In [9]:
uc, vc = func.calculate_windsFromStreamfunction_grid(psi_staggered_halo, dx_halo, dy_halo, dimensions, grid='C')

uc_2d = Quantity(uc, ('x', 'y_interface'), units['wind'], origins['compute_2d'], (dimensions['nx'], dimensions['ny1']), backend)
vc_2d = Quantity(vc, ('x_interface', 'y'), units['wind'], origins['compute_2d'], (dimensions['nx1'], dimensions['ny']), backend)

In [10]:
ud, vd = func.calculate_windsFromStreamfunction_grid(psi_halo, dx_halo, dy_halo, dimensions, grid='D')

ud_2d = Quantity(ud, ('x_interface', 'y'), units['wind'], origins['compute_2d'], (dimensions['nx1'], dimensions['ny']), backend)
vd_2d = Quantity(vd, ('x', 'y_interface'), units['wind'], origins['compute_2d'], (dimensions['nx'], dimensions['ny1']), backend)

#### 4. Prepare flux stencil

#### 4a. Extend initial conditions into the vertical

In [11]:
dimensions['nz'] = 1

empty = np.zeros((dimensions['nxhalo'], dimensions['nyhalo'], dimensions['nz']+1))
print(empty.shape)

qvapor_3d = np.copy(empty)
uc_3d = np.copy(empty)
vc_3d = np.copy(empty)
delp_3d = np.copy(empty)
qvapor_3d[:, :, 0] = qvapor_2d.data
uc_3d[:, :, 0] = uc_2d.data
vc_3d[:, :, 0] = vc_2d.data
delp_3d[:, :, 0] = delp_2d.data

qvapor = Quantity(qvapor_3d, ('x', 'y', 'z'), units['qvapor'], origins['compute_3d'], (dimensions['nx'], dimensions['ny'], dimensions['nz']), backend)
qvapor_init = Quantity(qvapor_3d, ('x', 'y', 'z'), units['qvapor'], origins['compute_3d'], (dimensions['nx'], dimensions['ny'], dimensions['nz']), backend)
uc = Quantity(uc_3d, ('x', 'y_interface', 'z'), units['wind'], origins['compute_3d'], (dimensions['nx'], dimensions['ny1'], dimensions['nz']), backend)
vc = Quantity(vc_3d, ('x_interface', 'y', 'z'), units['wind'], origins['compute_3d'], (dimensions['nx1'], dimensions['ny'], dimensions['nz']), backend)
delp = Quantity(delp_3d, ('x_interface', 'y', 'z'), units['wind'], origins['compute_3d'], (dimensions['nx1'], dimensions['ny'], dimensions['nz']), backend)

[stdout:0] (107, 107, 2)


[stdout:1] (107, 107, 2)


[stdout:2] (107, 107, 2)


[stdout:4] (107, 107, 2)


[stdout:3] (107, 107, 2)


[stdout:5] (107, 107, 2)


#### 4b. Prepare data for flux calculation

- use the function `FiniteVolumeFluxPrep`
    - requires `uc` and `vc` from (3c). 
    - requires some empty arrays to be filled in:
        - `crx` and `cry` are the dimensionless Courant numbers on C-grid points.
        - `xaf` and `yaf` are are fluxes of area in m2.
        - `ucc` and `vcc` are the contravariant velocities on C-grid points
        - `dt` is the acoustic time step.

In [12]:
%%capture capt
from fv3core.stencils.fxadv import FiniteVolumeFluxPrep

fvf_prep = FiniteVolumeFluxPrep(configuration['stencil_factory'], configuration['grid_data'])

crx = Quantity(empty, ('x_interface', 'y', 'z'), units['courant'], origins['compute_3d'], (dimensions['nx1'], dimensions['ny'], dimensions['nz']), backend)
cry = Quantity(empty, ('x', 'y_interface', 'z'), units['courant'], origins['compute_3d'], (dimensions['nx'], dimensions['ny1'], dimensions['nz']), backend)

xaf = Quantity(empty, ('x_interface', 'y', 'z'), units['areaflux'], origins['compute_3d'], (dimensions['nx1'], dimensions['ny'], dimensions['nz']), backend)
yaf = Quantity(empty, ('x', 'y_interface', 'z'), units['areaflux'], origins['compute_3d'], (dimensions['nx'], dimensions['ny1'], dimensions['nz']), backend)

ucv = Quantity(empty, ('x_interface', 'y', 'z'), units['wind'], origins['compute_3d'], (dimensions['nx1'], dimensions['ny'], dimensions['nz']), backend)
vcv = Quantity(empty, ('x', 'y_interface', 'z'), units['wind'], origins['compute_3d'], (dimensions['nx'], dimensions['ny1'], dimensions['nz']), backend)


dt_acoustic = 300.

fvfp = fvf_prep(uc, vc, crx, cry, xaf, yaf, ucv, vcv, dt_acoustic)

print()
print('UC min, mean, max: %.2f %.2f %.2f' % (np.nanmin(uc), np.nanmean(uc), np.nanmax(uc)))
print('UCV min, mean, max: %.2f %.2f %.2f' % (np.nanmin(ucv), np.nanmean(ucv), np.nanmax(ucv)))
print()
print('VC min, mean, max: %.2f %.2f %.2f' % (np.nanmin(vc), np.nanmean(vc), np.nanmax(vc)))
print('VCV min, mean, max: %.2f %.2f %.2f' % (np.nanmin(vcv), np.nanmean(vcv), np.nanmax(vcv)))
print()
print('X-Courant min, mean, max: %.2f %.2f %.2f' % (np.nanmin(crx), np.nanmean(crx), np.nanmax(crx)))
print('Y-Courant min, mean, max: %.2f %.2f %.2f' % (np.nanmin(cry), np.nanmean(cry), np.nanmax(cry)))
print()
print('X-area flux min, mean, max: %.2f %.2f %.2f' % (np.nanmin(xaf), np.nanmean(xaf), np.nanmax(xaf)))
print('X-area flux min, mean, max: %.2f %.2f %.2f' % (np.nanmin(yaf), np.nanmean(yaf), np.nanmax(yaf)))

#### (4c) Create finite volume transport stencil

- use the function `FiniteVolumeTransport` 
    - takes in the tracer `qvap` to be transported.
    - takes in the `crx` and `cry` Courant numbers, `xaf` and `yaf` area fluxes from previous step.
    - requires some empty arrays to be filled in:
        - `qfx` and `qfy` are the output fluxes of the tracer to be transported -- i think kg/m2?
        
- also takes in mass fluxes (optional) -- where would I find those?

- do I actually need the `qfx`, `qfy` that are output? Yes - for updating mass fluxes

In [13]:
#%%capture capt
from fv3core.stencils.fvtp2d import FiniteVolumeTransport
import copy as cp

grid_type = 0
hord = 6
dt = 1800.
density = 1

fvtp2d = FiniteVolumeTransport(configuration['stencil_factory'], configuration['grid_data'], configuration['damping_coefficients'], grid_type, hord)


mfxd = Quantity(empty, ('x_interface', 'y', 'z'), units['wind'], origins['compute_3d'], (dimensions['nx1'], dimensions['ny'], dimensions['nz']), backend)
mfyd = Quantity(empty, ('x', 'y_interface', 'z'), units['wind'], origins['compute_3d'], (dimensions['nx'], dimensions['ny1'], dimensions['nz']), backend)

mfxd.data[:] = xaf.data[:] * delp.data[:] * density
mfyd.data[:] = yaf.data[:] * delp.data[:] * density


In [14]:
# mfxd_global = configuration['communicator'].gather(mfxd)
# mfyd_global = configuration['communicator'].gather(mfyd)
# if mpi_rank == 0:

#     mfxd_global = func.unstagger_coord(mfxd_global.data[:, :, :, 0])
#     mfyd_global = func.unstagger_coord(mfyd_global.data[:, :, :, 0])
#     variables = {'lon': lon_global.data, 'lat': lat_global.data, 'mfxd': mfxd_global.data[:, :, :, 0], 'mfyd': mfyd_global.data[:, :, :, 0]}

#     #func.plot_projection_field(variables['lon'], variables['lat'], mfyd_global, vmin=-1e11, vmax=1e11, cmap='bwr', title='mfxd')

In [15]:
# #%%capture capt
# from fv3core.stencils.fvtp2d import FiniteVolumeTransport
# import copy as cp

# grid_type = 0
# hord = 6
# dt = 1800.

# delp.data[:] += qvapor.data[:]

# qfx = Quantity(empty, ('x_interface', 'y', 'z'), units['wind'], origins['compute_3d'], (dimensions['nx1'], dimensions['ny'], dimensions['nz']), backend)
# qfy = Quantity(empty, ('x', 'y_interface', 'z'), units['wind'], origins['compute_3d'], (dimensions['nx'], dimensions['ny1'], dimensions['nz']), backend)

# fvtp2d = FiniteVolumeTransport(configuration['stencil_factory'], configuration['grid_data'], configuration['damping_coefficients'], grid_type, hord)
# fvTransport = fvtp2d(qvapor, crx, cry, xaf, yaf, qfx, qfy, mass=delp)




In [16]:
from fv3core.stencils.tracer_2d_1l import TracerAdvection

tracers = {'qvapor': qvapor}
tracers_init = {'qvapor': qvapor_init}
tracAdv = TracerAdvection(configuration['stencil_factory'], fvtp2d, configuration['grid_data'], configuration['communicator'], tracers)

#mfxd = Quantity(empty, ('x_interface', 'y', 'z'), units['pressure'], origins['compute_3d'], (dimensions['nx1'], dimensions['ny'], dimensions['nz']), backend)
#mfyd = Quantity(empty, ('x', 'y_interface', 'z'), units['pressure'], origins['compute_3d'], (dimensions['nx'], dimensions['ny1'], dimensions['nz']), backend)
#delp_tmp = Quantity(empty, ('x', 'y', 'z'), units['pressure'], origins['compute_3d'], (dimensions['nx'], dimensions['ny'], dimensions['nz']), backend)

# mfxd.data[:] += (xaf.data[:] * delp.data[:] * dt/dt_acoustic)
# mfyd.data[:] += (yaf.data[:] * delp.data[:] * dt/dt_acoustic)
# delp_tmp.data[1:-1, 1:-1, 0] += delp.data[1:-1, 1:-1, 0] + (mfxd.data[1:-1, 1:-1, 0] - mfxd.data[2:, 1:-1, 0] + mfyd.data[1:-1, 1:-1, 0] - mfyd.data[1:-1, 2:, 0]) * configuration['grid_data'].rarea.data[1:-1, 1:-1]
# delp_tmp.data[:, 0, :]= delp_tmp.data[:, 1, :]
# delp_tmp.data[:, -1, :]= delp_tmp.data[:, -2, :]
# delp_tmp.data[0, :, :] = delp_tmp.data[1, :, :]
# delp_tmp.data[-1, :, :] = delp_tmp.data[-2, :, :]
# mfxd.data[:] += qfx.data[:]
# mfyd.data[:] += qfy.data[:]



In [17]:
tracers['qvapor'].data.shape

Out[1:15]: (107, 107, 2)

Out[5:15]: (107, 107, 2)

Out[2:15]: (107, 107, 2)

Out[4:15]: (107, 107, 2)

Out[0:15]: (107, 107, 2)

Out[3:15]: (107, 107, 2)

#### 5. Perform tracer advection -- no updates

In [18]:
#%%capture capt

nSteps_adv = 30

crx_init = cp.deepcopy(crx)
cry_init = cp.deepcopy(cry)
mfxd_init = cp.deepcopy(mfxd)
mfyd_init = cp.deepcopy(mfyd)
delp_init = cp.deepcopy(delp)
diff_minMax = []
tracers_store = []
for step in range(nSteps_adv):
    tmp = cp.deepcopy(tracers) # get pre-step value

    tracAdv(tracers, delp, mfxd, mfyd, crx, cry, dt)

    delp = cp.deepcopy(delp_init)
    mfxd = cp.deepcopy(mfxd_init)
    mfyd = cp.deepcopy(mfyd_init)
    crx = cp.deepcopy(crx_init)
    cry = cp.deepcopy(cry_init)
    tracers_store.append(cp.deepcopy(tracers['qvapor']))



    diff = tracers['qvapor'].data - tmp['qvapor'].data
    diff_init = tracers['qvapor'].data - tracers_init['qvapor'].data
    diff_minMax.append('step %3s: timestep diff min = %.2e; max = %.2e; init diff min = %.2e; max = %.2e' % (step+1, np.nanmin(diff), np.nanmax(diff), np.nanmin(diff_init), np.nanmax(diff_init)))

    # if mpi_rank == 0 and (step)%10 == 0:
    #     qvapor_global = configuration['communicator'].gather(tracers['qvapor'])
    #     variables = {'lon': lon_global.data, 'lat': lat_global.data, 'q': qvapor_global.data}
    #     func.plot_projection_field(variables['lon'], variables['lat'], variables['q'], vmin=0, vmax=1, cmap='viridis', title='q_vapor step %s' % step, units=units['qvapor'])


if mpi_rank == 0: 
    print()
    print('\n'.join(diff_minMax))

[stdout:0] 
step   1: timestep diff min = -6.84e-02; max = 6.84e-02; init diff min = -6.84e-02; max = 6.84e-02
step   2: timestep diff min = -6.84e-02; max = 6.84e-02; init diff min = -1.37e-01; max = 1.36e-01
step   3: timestep diff min = -6.83e-02; max = 6.84e-02; init diff min = -2.04e-01; max = 2.04e-01
step   4: timestep diff min = -6.84e-02; max = 6.83e-02; init diff min = -2.71e-01; max = 2.70e-01
step   5: timestep diff min = -6.84e-02; max = 6.83e-02; init diff min = -3.35e-01; max = 3.36e-01
step   6: timestep diff min = -6.85e-02; max = 6.84e-02; init diff min = -4.00e-01; max = 3.99e-01
step   7: timestep diff min = -6.85e-02; max = 6.84e-02; init diff min = -4.61e-01; max = 4.60e-01
step   8: timestep diff min = -6.84e-02; max = 6.85e-02; init diff min = -5.21e-01; max = 5.21e-01
step   9: timestep diff min = -6.83e-02; max = 6.85e-02; init diff min = -5.78e-01; max = 5.77e-01
step  10: timestep diff min = -6.84e-02; max = 6.84e-02; init diff min = -6.32e-01; max = 6.33e-0

Tracer advection -- updating fluxes?

In [19]:
lona =  Quantity(configuration['grid_data'].lon_agrid.data * 180 / np.pi, ('x_interface', 'y_interface'), units['coord'], origins['compute_2d'], (dimensions['nx1'], dimensions['ny1']), backend)
lata =  Quantity(configuration['grid_data'].lat_agrid.data * 180 / np.pi, ('x_interface', 'y_interface'), units['coord'], origins['compute_2d'], (dimensions['nx1'], dimensions['ny1']), backend)
lon_global = configuration['communicator'].gather(lona)
lat_global = configuration['communicator'].gather(lata)


In [ ]:

if mpi_rank == 0:
    variables = {'lon': lon_global.data, 'lat': lat_global.data, 'q_start': qvapor_start.data[:, :, :, 0], 'q_end': qvapor_end.data[:, :, :, 0]}
    func.plot_projection_field(variables['lon'], variables['lat'], variables['q_start'], vmin=-0, vmax=1, cmap='viridis', title='q_vapor init', units=units['qvapor'])
    func.plot_projection_field(variables['lon'], variables['lat'], variables['q_end'], vmin=-0, vmax=1, cmap='viridis', title='q_vapor after tracerAdvection', units=units['qvapor'])
    func.plot_projection_field(variables['lon'], variables['lat'], (variables['q_end'] - variables['q_start']), vmin=-1e0, vmax=1e0, cmap='bwr', title='q_vapor after - before tracerAdvection', units='kg/kg')

Out[1:30]: <module 'functions' from '/home/ajdas/pace/advection_test/functions.py'>

Out[3:30]: <module 'functions' from '/home/ajdas/pace/advection_test/functions.py'>

Out[2:30]: <module 'functions' from '/home/ajdas/pace/advection_test/functions.py'>

Out[5:30]: <module 'functions' from '/home/ajdas/pace/advection_test/functions.py'>

Out[0:30]: <module 'functions' from '/home/ajdas/pace/advection_test/functions.py'>

Out[4:30]: <module 'functions' from '/home/ajdas/pace/advection_test/functions.py'>

In [35]:
importlib.reload(func)
qvapor_global = []
for step in range(len(tracers_store)):
    qvapor_global.append(configuration['communicator'].gather(tracers_store[step]))

saveDir = '/home/ajdas/pace/advection_test/figs/'
fOut = saveDir + 'advection_step_'

if mpi_rank == 0:
    qvapor_global = np.stack(qvapor_global)[:, :, :, :, 0]
    variables = {'lon': lon_global.data, 'lat': lat_global.data, 'q': qvapor_global}

    #for step in range(len(tracers_store)):
    for step in range(1):
        func.plot_projection_field(variables['lon'], variables['lat'], variables['q'][step], vmin=-0, vmax=1, cmap='viridis', title='q_vapor advection step %03d' % step, units=units['qvapor'], fSave=fOut+'{:03d}'.format(step) + '.png')


%px:   0%|          | 0/6 [00:00<?, ?tasks/s]

In [26]:
tmp = np.stack(qvapor_global)

In [27]:
tmp.shape

Out[0:25]: (30, 6, 101, 101, 1)

Out[2:25]: (30,)

Out[4:25]: (30,)

Out[1:25]: (30,)

Out[5:25]: (30,)

Out[3:25]: (30,)

In [30]:
save = '3'

In [31]:
if save not None:
    print(save)

SyntaxError: invalid syntax (798931604.py, line 1)

In [ ]:
%autopx

In [ ]:
cluster.shutdown()